In [1]:
import pandas as pd
import numpy as np

In [2]:
# Let's practise
# I want to test whether large firms would have lower returns

In [3]:
asset = pd.read_csv('https://mfin6201.s3.amazonaws.com/company_assets.csv')

In [4]:
mret = pd.read_csv('https://mfin6201.s3.amazonaws.com/monthly_return.csv')

In [5]:
# let's inspect monthly return dataset
mret

,permno,date,fyear,ret
0,10015.00,1985-12-31,1985,0.07
1,10031.00,1985-12-31,1985,-0.01
2,10057.00,1985-12-31,1985,-0.06
3,10065.00,1985-12-31,1985,0.04
4,10066.00,1985-12-31,1985,-0.14
...,...,...,...,...
3650265,93426.00,2018-12-31,2018,-0.11
3650266,93427.00,2018-12-31,2018,-0.03
3650267,93429.00,2018-12-31,2018,-0.09
3650268,93434.00,2018-12-31,2018,-0.34


#### The objective is to aggregate monthly returns to annual returns
#### In MFIN6201, we simply take the average; this time, let's do this more carefully.

$ret_{t,t+n} = (1+ret_t)\times(1+ret_{t+1})\times(1+ret_{t+2}) ... \times(1+ret_{t+n}) - 1$

In [6]:
mret['ret'] = mret['ret'] + 1

In [7]:
# Let's use groupby to calculate the products
aret = mret.groupby(['permno','fyear']).ret.prod()-1

In [8]:
aret = aret.reset_index()

In [9]:
# now we can merge with asset
asset.merge(aret)
# seems OK, but there may be duplicates in one of the dataset that rendering this non-panel
merged = asset.merge(aret)

In [10]:
# I will define a function to check if the data is panel
def check_panel(df,f,t):
    check_point = df.groupby([f,t]).size()[lambda s:s>1]
    assert len(check_point)==0, "You do not have a Panel..."        

In [11]:
check_panel(merged,'permno','fyear')

AssertionError: You do not have a Panel...

In [ ]:
# In the merging process, you can set a validator to make sure you do not have duplicates
asset.merge(aret,validate='1:1')

In [ ]:
asset = asset.drop_duplicates(['permno','fyear'])

In [ ]:
# In the merging process, you can set a validator to make sure you do not have duplicates
merged = asset.merge(aret,validate='1:1')

In [ ]:
merged['ln_at'] = np.log(merged['at']+1)

In [ ]:
merged

In [ ]:
from statsmodels.regression.linear_model import OLS
from linearmodels.panel import PanelOLS

In [ ]:
#               y,             x
res = OLS(merged['ret'],merged['ln_at']).fit()

In [ ]:
res.summary()

In [ ]:
# seems we have outliers
merged['ret'].hist()

In [ ]:
from scipy.stats.mstats import winsorize

In [ ]:
merged['ret'] = winsorize(merged['ret'],limits=0.01)

In [ ]:
# seems we have outliers
merged['ret'].hist() # better

In [ ]:
res = OLS(merged['ret'],merged['ln_at']).fit()

In [ ]:
res.get_robustcov_results().summary2()

In [ ]:
merged = merged.set_index(['permno','fyear'])

In [ ]:
res = PanelOLS(merged['ret'],merged['ln_at'],entity_effects=True,time_effects=True).fit()

In [ ]:
res.summary

In [ ]:
res = PanelOLS(merged['ret'],merged['ln_at'],entity_effects=True,time_effects=True).fit(cov_type='clustered',cluster_entity=True)

In [ ]:
res.summary